# Pre-requisite  

In [ ]:
!pip install --upgrade -q boto3 langchain sagemaker-studio-image-build aws-sam-cli

# Workshop variables

In [ ]:
name="gen-ai-workshop"
jumpstart_model = "huggingface-llm-falcon-7b-instruct-bf16"
endpoint_name=f"{name}-endpoint"

In [ ]:
import boto3


aws_region = boto3.Session().region_name
aws_account_id = boto3.client('sts').get_caller_identity().get('Account')

# Launch Kendra Index

Kendra will be the RAG endpoint, that will store our documents, for RAG prompt engineering.

We will first create a role for Kendra

In [ ]:
import boto3
import json

iam_client = boto3.client('iam')

kendra_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "kendra.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
        }]
    }

policy_cloudwatch_arn = "arn:aws:iam::aws:policy/CloudWatchLogsFullAccess"

try:
    response =iam_client.get_role(RoleName=f"kendra-{name}-role")
    print(f"kendra-{name}-role Role already exists")
except:
    response = iam_client.create_role(
        RoleName=f"kendra-{name}-role",
        AssumeRolePolicyDocument=json.dumps(kendra_trust_policy),
    )
    print(f"Created the role kendra-{name}-role")
    
kendra_role_arn = response["Role"]["Arn"]

try:
    response = iam_client.attach_role_policy(
        RoleName=f"kendra-{name}-role",
        PolicyArn=policy_cloudwatch_arn
    )
except:
    print(f"Policy already attached to role kendra-{name}-role")
    
    


In [ ]:
kendra_client = boto3.client("kendra")

def list_kendra_indices(kendra_client):
    kendra_indices = kendra_client.list_indices()
    return kendra_indices["IndexConfigurationSummaryItems"]


def find_index_id_according_to_tag(kendra_client, indices, aws_region, aws_account_id):
    for k_index in indices:
        describe_index_tags_response = kendra_client.list_tags_for_resource(
            ResourceARN=f"arn:aws:kendra:{aws_region}:{aws_account_id}:index/{k_index['Id']}"
        )
        for tag in describe_index_tags_response["Tags"]:
            if tag["Key"] == "workshop" and tag["Value"] == "gen-ai":
                return k_index['Id']
    return None


def create_kendra_index(kendra_client):
    kendra_index_response = kendra_client.create_index(
        Name=f"{name}-index",
        Edition="DEVELOPER_EDITION",
        RoleArn=kendra_role_arn,
        Tags=[{
            'Key': 'workshop',
            'Value': 'gen-ai'
            }]
    )
    kendra_index_id = kendra_index_response["Id"]
    return kendra_index_id


# Creating Kendra index
kendra_indices = list_kendra_indices(kendra_client)
if len(kendra_indices) > 0:
    kendra_index_id = find_index_id_according_to_tag(kendra_client, kendra_indices, aws_region, aws_account_id)
    if kendra_index_id is None:
        # you have indices but not tagged with workshop=gen-ai
        kendra_index_id = create_kendra_index(kendra_client)
else:
    create_kendra_index(kendra_client)


print(kendra_index_id)


## LLM Endpoint provisioning

Now we will deploy LLM Model Falcon 7B instruct using SageMaker sdk `JumpstartModel` class, that will do all the heavy lifting configuring the endpoint in Amazon SageMaker.

In [ ]:
from sagemaker import get_execution_role


try:
    sm_execution_role = get_execution_role()
except:
    # To work locally use explicit role
    sm_execution_role = "arn:aws:iam::910416587115:role/SageMaker-Role-Full"

print(sm_execution_role)

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel


jumpstart_model_id = "huggingface-textgeneration-falcon-7b-instruct-bf16"
sagemaker_endpoint_name = f"{name}-falcon-7b-instruct"


try:
    model = JumpStartModel(model_id=jumpstart_model_id, role=sm_execution_role)
    model.deploy(endpoint_name=f"{name}-falcon-7b-instruct-1", wait=False)
except Exception as e:
    print(e)
    print(f"""\nPlease make sure that you dont have in your account an endpoint or endpoint configuration with name {sagemaker_endpoint_name}\n
          Endpoint configuration: Check at https://{aws_region}.console.aws.amazon.com/sagemaker/home?region={aws_region}#/endpointConfig
          Endpoint: Check at https://{aws_region}.console.aws.amazon.com/sagemaker/home?region={aws_region}#/endpoints/ 
          
          If the endpoint is already running, you may continue the workshop and use it.
          """)
    
    

# Build the backend lambda, and API Gateway

By now, we have launch Amazon Kendra, and Falcon LLM, using Amazon SageMaker endpoint.

Now we will build the Backend lambda, using [AWS Serverless Application Model](https://aws.amazon.com/serverless/sam/) (SAM), an open-source framework for building serverless applications.

The lambda code [rag_app](/lab4/rag_app/) contains couple of environment variables that help us control the lambda behavior.


## Build the container image for frontend chatbot application

While using Sagemaker studio, using `sagemaker-studio-image-build` we can trigger a docker build leveraging [AWS CodeBuild](https://aws.amazon.com/codebuild/)

We will start by adding appropriate roles to SageMaker execution role, to allow triggering the build job.

In [ ]:
import boto3

sm_execution_role_name = sm_execution_role.split("/")[-1]

with open("codebuild-policy.json") as f:
    code_build_policy_document = f.read()

iam_client = boto3.client("iam")

try:
    policy_response = iam_client.create_policy(
        PolicyName=f"codebuild-policy-sm-docker-build",
        PolicyDocument=code_build_policy_document
    )
    code_build_policy_arn = policy_response['Policy']['Arn']
except:
    print("Policy exists")
    code_build_policy_arn = f"arn:aws:iam::{aws_account_id}:policy/codebuild-policy-sm-docker-build"
    

attach_response = iam_client.attach_role_policy(
    RoleName=sm_execution_role_name,
    PolicyArn=code_build_policy_arn
)

codebuild_trust_policy = {
     "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "codebuild.amazonaws.com" 
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

response = iam_client.update_assume_role_policy(
    RoleName=sm_execution_role_name,
    PolicyDocument=json.dumps(codebuild_trust_policy)
)

Now that we have configured SageMaker execution role, we can trigger a build job to build the frontend chatbot application that was built using [`streamlit`](https://streamlit.io/).

In [ ]:
!cd fe && sm-docker build --role $sm_execution_role_name --repository gen-ai-streamlit-fe:latest .

Now we will update the [template](/lab4/template.yml), and deploy the stack using sam-cli

In [140]:
with open("template.yml", 'r') as f:
    template = f.read()
    
update_template = template.replace("***KENDRA_INDEX_ID***", f"{kendra_index_id}")

with open("template.yml", 'w') as f:
    f.write(update_template)

Now that we have updated the template file lets verify that all the components are ready, and deploy the stack

In [142]:
!sam build

Building codeuri:                                                               
/Users/omerhaim/work/aws/generative-ai-on-aws-immersion-day/lab4/rag_app        
runtime: python3.10 metadata: {} architecture: x86_64 functions: RagAppFunction 
Running PythonPipBuilder:ResolveDependencies                                    
Running PythonPipBuilder:CopySource                                             

Build Succeeded

Built Artifacts  : .aws-sam/build
Built Template   : .aws-sam/build/template.yaml

Commands you can use next
[*] Validate SAM template: sam validate
[*] Invoke Function: sam local invoke
[*] Test Function in the Cloud: sam sync --stack-name {{stack-name}} --watch
[*] Deploy: sam deploy --guided


In [143]:
!sam deploy --stack-name gen-ai-immersion-day-stack --resolve-s3 --capabilities CAPABILITY_IAM


		Managed S3 bucket: aws-sam-cli-managed-default-samclisourcebucket-ppxavannbzh9
		A different default S3 bucket can be set in samconfig.toml
		Or by specifying --s3-bucket explicitly.
	Uploading to ef6b4b240b79ae7bdbc18c797f1c6b7f  39921051 / 39921051  (100.00%)

	Deploying with following values
	Stack name                   : gen-ai-immersion-day-stack
	Region                       : eu-west-1
	Confirm changeset            : False
	Disable rollback             : False
	Deployment s3 bucket         : aws-sam-cli-managed-default-samclisourcebucket-ppxavannbzh9
	Capabilities                 : ["CAPABILITY_IAM"]
	Parameter overrides          : {}
	Signing Profiles             : {}

Initiating deployment

	Uploading to dba9fec7897bcb45e84beca1e0ae8c0e.template  9694 / 9694  (100.00%)


Waiting for changeset to be created..

CloudFormation stack changeset
-------------------------------------------------------------------------------------------------
Operation                LogicalResou

In [147]:
!aws elbv2 describe-load-balancers --query 'LoadBalancers[?LoadBalancerName==`rag-load-balancer`].[DNSName]' --output text

rag-load-balancer-881140808.eu-west-1.elb.amazonaws.com
